In [1]:
from satelite_data_preporation import collection
%load_ext autoreload
%autoreload 2

In [2]:
region_boundary = 'vector_data/budenovsk_district_boundaries.geojson'
region_of_interest = 'vector_data/our_fields_for_animation.geojson'



In [3]:
sentinel2021_collection = collection('sentinel2', '2019-08-15', '2019-09-01', region_boundary, region_of_interest)

In [7]:
sentinel2021_collection.DownloadImages()

Generating URL ...
Please wait ...
Data downloaded to c:\Users\User\Google Диск\python_notebooks\total_archangelskiy\field_scale\raster_data\Field_scale_sentinel2_2019-08-18_scene.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\User\Google Диск\python_notebooks\total_archangelskiy\field_scale\raster_data\Field_scale_sentinel2_2019-08-28_scene.tif


{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:32638',
   'crs_transform': [60, 0, 399960, 0, -60, 5100000]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:32638',
   'crs_transform': [10, 0, 399960, 0, -10, 5100000]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:32638',
   'crs_transform': [10, 0, 399960, 0, -10, 5100000]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:32638',
   'crs_transform': [10, 0, 399960, 0, -10, 5100000]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'crs': 'EPSG:32638',
   'crs_transform': [20, 0, 399960, 0, -20, 5100000]},
  {'id': 'B6',
   'data_type

{'type': 'ImageCollection',
 'bands': [],
 'features': [{'type': 'Image',
   'bands': [{'id': 'B1',
     'data_type': {'type': 'PixelType',
      'precision': 'int',
      'min': -32768,
      'max': 32767},
     'dimensions': [123, 134],
     'origin': [4520, 3873],
     'crs': 'EPSG:32638',
     'crs_transform': [30, 0, 296685, 0, -30, 5050515]},
    {'id': 'B2',
     'data_type': {'type': 'PixelType',
      'precision': 'int',
      'min': -32768,
      'max': 32767},
     'dimensions': [123, 134],
     'origin': [4520, 3873],
     'crs': 'EPSG:32638',
     'crs_transform': [30, 0, 296685, 0, -30, 5050515]},
    {'id': 'B3',
     'data_type': {'type': 'PixelType',
      'precision': 'int',
      'min': -32768,
      'max': 32767},
     'dimensions': [123, 134],
     'origin': [4520, 3873],
     'crs': 'EPSG:32638',
     'crs_transform': [30, 0, 296685, 0, -30, 5050515]},
    {'id': 'B4',
     'data_type': {'type': 'PixelType',
      'precision': 'int',
      'min': -32768,
      'ma

Теперь нужно допилить систему чтоб у нас два раза не скачивались одни и теже данные , и чтобы система работала на всех платформах

In [1]:
import os.path 

In [26]:
os.path.isfile('raster_data/Field_scale_landsat7_2001-10-26_scene.tif')

False